In [ ]:
import csv
from io import StringIO
import pyperclip

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt

import csv
from io import StringIO
import pyperclip

from tabulate import tabulate
printtab = lambda x : print(tabulate(x, headers='firstrow'))


In [ ]:
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-07-17_11-54-40-709189"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-07-17_11-55-46-512820"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-07-22_18-32-29-456924"
output_dir = os.path.join(base_dir, "LYL_outputs")
outputs_dirs = [os.path.join(output_dir, f"option_{i}") for i in range(6)]
outputs_dirs = outputs_dirs[:3] + outputs_dirs[4:] # because we removed ban option 3

In [ ]:
# NUM_PEOPLE_IN_COHORT = 230790379.40376273
cohort_size_by_demo = {
    0: 230454895.2065651,
    1: 28100330.965297583,
    2: 202354564.2412675,
    3: 50679971.42431204,
    4: 179774923.78225303,
}

In [ ]:
opt0_dir = outputs_dirs[0]
print(outputs_dirs)

In [ ]:
print(opt0_dir)

In [ ]:
# Grab the LYL data
all_data_all_options = []
for opt in (0,1,2,3,4):
    opt_dir = outputs_dirs[opt]
    all_data = []
    for f in glob(opt_dir + "/*.npy"):
        arr = np.load(f) 
        all_data.append(arr)
    if all_data != []:
        all_data_all_options.append(all_data)

all_data_all_options = np.array(all_data_all_options)

print(all_data_all_options.shape) # ban options, runs, demographics
print(all_data_all_options[0,0,0])



In [ ]:
# Define the demographic groups
demo_names = [
    "all demographics",
    "black",
    "nonblack",
    "pov",
    "nonpov",
]

option_names = [
    "SQ",
    "B1",
    "B2",
    "B3",
    "B4",
]

In [ ]:
def get_mean_lower_upper(arr, axis, decimals):
    """
    Given an array, return the mean across an axis, as well as the lower and upper bounds of a 95% confidence interval
    Round outputs to the nearest tenth
    """
    return (
        np.around(np.mean(arr, axis=axis),decimals=decimals),
        np.around(np.percentile(arr, 2.5, axis=axis),decimals=decimals),
        np.around(np.percentile(arr, 97.5, axis=axis),decimals=decimals),
    )


def LYL_table(adao_LYL, averaging=False):
    counts = np.copy(adao_LYL)
    if averaging:
        counts[:,:,0] /= cohort_size_by_demo[0]
        counts[:,:,1] /= cohort_size_by_demo[1]
        counts[:,:,2] /= cohort_size_by_demo[2]
        counts[:,:,3] /= cohort_size_by_demo[3]
        counts[:,:,4] /= cohort_size_by_demo[4]

    all_sq = np.tile(counts[0,:,:],(counts.shape[0],1,1))
    assert (all_sq.shape == counts.shape)
    change = (counts - all_sq) / all_sq
    change *= 100 # for percentages

    # expected = change < 0
    # unexpected = change > 1
    # print(np.count_nonzero(unexpected))
    # print(np.count_nonzero(expected))


    # get mean, upper, lower
    if not averaging:
        counts_mean, counts_lower, counts_upper = get_mean_lower_upper(counts, axis=1, decimals=-1)
        change_mean, change_lower, change_upper = get_mean_lower_upper(change, axis=1, decimals=3)
    else:
        counts_mean, counts_lower, counts_upper = get_mean_lower_upper(counts, axis=1, decimals=4)
        change_mean, change_lower, change_upper = get_mean_lower_upper(change, axis=1, decimals=3)

    # now the shape is options, demos

    # print table
    # the rows are demographic groups, columns are ban options.
    # need count, percentage, and %change (for ban options)

    header = ["Demographic"]
    for i, option_name in enumerate(option_names):
        if i > counts.shape[0] - 1: break
        header += [f"{option_name} Count"]
        if (i != 0):
            header += [f"{option_name} % Change from SQ"]

    rows= []
    for i, demo_name in enumerate(demo_names):
        row = [f"{demo_name}"]
        for j, _ in enumerate(option_names):
            if j > counts.shape[0] - 1: break
            row += [f"{counts_mean[j,i]} ({counts_lower[j,i]}, {counts_upper[j,i]})"]
            if (j != 0):
                row += [f"{change_mean[j,i]}% ({change_lower[j,i]}, {change_upper[j,i]})"]

                # change_mean = (counts_mean[j,i] - counts_mean[0,i]) / counts_mean[0,i]
                # change_mean = np.around(change_mean * 100, decimals=3)
                # row += [f"{change_mean}%"]

        rows += [row]

    table = [header] + rows
    print(tabulate(table, headers='firstrow', tablefmt='grid'))

    output = StringIO()
    writer = csv.writer(output)
    writer.writerow(header)  # Write headers
    writer.writerows(rows)  # Write rows
    csv_output = output.getvalue()
    print(csv_output)
    pyperclip.copy(csv_output)  # Now paste into Excel!

In [ ]:
LYL_table(all_data_all_options)

In [ ]:
LYL_table(all_data_all_options, averaging=True)